# 辞書とAIモデルをコピーし、jarファイルを作成する

In [1]:
import os
with open('../version.txt','r',encoding='utf-8') as f:
    for l in f:
        version=l.rstrip()
        break
os.environ["VER"]=version


In [2]:
!rm -rf target

In [3]:
!echo $VER

1.7o


In [4]:
!rm -rf src/main/resources/ai/*

In [5]:
!rm -rf src/main/resources/ai_r/*

In [6]:
!rm -rf src/main/resources/dict/*

In [7]:
import glob
import shutil
for fname in glob.glob(f'../train/model/model.{version}/*.pt'):
    if "checkpoint" in fname:
        continue

    shutil.copy(fname,'src/main/resources/ai/')


In [8]:
for fname in glob.glob(f'../train/model/model.{version}/*.txt'):
    if fname.startswith("checkpoint"):
        continue

    shutil.copy(fname,'src/main/resources/ai/')


In [9]:
for fname in glob.glob(f'../train/model/model.{version}/*.json'):
    if fname.startswith("checkpoint"):
        continue

    shutil.copy(fname,'src/main/resources/ai/')

In [10]:
for fname in glob.glob(f'../train/model/model_r.{version}/*.pt'):
    if "checkpoint" in fname:
        continue

    shutil.copy(fname,'src/main/resources/ai_r/')

In [11]:
for fname in glob.glob(f'../train/model/model_r.{version}/*.txt'):
    if fname.startswith("checkpoint"):
        continue

    shutil.copy(fname,'src/main/resources/ai_r/')

In [12]:
for fname in glob.glob(f'../train/model/model_r.{version}/*.json'):
    if fname.startswith("checkpoint"):
        continue

    shutil.copy(fname,'src/main/resources/ai_r/')

In [13]:
for fname in glob.glob(f'../dict/data/data.{version}/*json'):
    shutil.copy(fname,'src/main/resources/dict/')

In [14]:
with open("kanjikana.properties",'r',encoding='utf-8') as f:
    lst = [l.rstrip() for l in f]


newlst=[]
for l in lst:
    newlst.append(l.replace(r"${project.version}",version))

with open("src/main/resources/kanjikana.properties",'w',encoding='utf-8') as f:
    for l in newlst:
        f.write(l+"\n")

In [15]:
!cat pom.xml.template |sed s/VER/$VER/ > pom.xml 
!mvn clean compile assembly:single

!mkdir -p lib
!mv target/*jar lib

[INFO] Scanning for projects...
[INFO] 
[INFO] ---------------< jp.go.digital.kanjikana:kanjikana_core >---------------
[INFO] Building kanjikana_core 1.7o
[INFO] --------------------------------[ jar ]---------------------------------
Downloaded from central: https://repo.maven.apache.org/maven2/ai/djl/api/0.25.0/api-0.25.0.pom (1.9 kB at 2.1 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/net/java/dev/jna/jna/5.13.0/jna-5.13.0.pom (2.0 kB at 18 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/ai/djl/pytorch/pytorch-engine/0.25.0/pytorch-engine-0.25.0.pom (1.4 kB at 2.0 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/net/java/dev/jna/jna/5.13.0/jna-5.13.0.jar (1.9 MB at 4.1 MB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/ai/djl/api/0.25.0/api-0.25.0.jar (884 kB at 1.3 MB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/ai/djl/pytorch/pytorch-engine/0.25.0/pytorch-engine-0.25.0.jar (96 kB at 

In [16]:
!date

2025年 10月  6日 月曜日 16:32:47 JST
